- Verifique se há valores ausentes nos metadados e como esses casos são tratados.
- Verifique valores inconsistentes, por exemplo: dimensões de imagens fora do esperado.

In [ ]:
import os
import pandas as pd
from tabulate import tabulate

CSV_PATH = r"C:\Users\luana\Desktop\Squad04-Semantic-Segmentation-Drone-Dataset\dataset\binary_dataset\binary_dataset\metadata_binario.csv"
VALORES_INVALIDOS = {"", "N/A", "None", "[]", "{}", "nan", "NaN", "n/a", "null"}

CLASSES_BINARIAS = {
    0: {0, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20},  # background
    1: {1, 2, 3, 4, 9}  # objeto
}

# Função para formatar porcentagens de acordo com as faixas de pixels
# de background e objeto
def formatar_porcentagem(valor):
    if valor == 0:
        return "0%"
    elif valor < 0.01:
        return "<1%"
    elif valor < 0.1:
        return "1-10%"
    elif valor < 0.25:
        return "10-25%"
    elif valor < 0.5:
        return "25-50%"
    elif valor < 0.75:
        return "50-75%"
    else:
        return ">75%"

def verificar_metadados(csv_path):
    if not os.path.exists(csv_path):
        print(f"[ERRO] Arquivo não encontrado: {csv_path}")
        return

    df = pd.read_csv(csv_path)
    total_imagens = len(df)
    print(f"\nℹ️ Total de imagens no dataset: {total_imagens:,}")

    print("\n 1. Valores ausentes ou inválidos:")
    print("-" * 50)
    nulos = df.isnull().sum()
    nulos = nulos[nulos > 0]

    if nulos.empty:
        print(" Nenhum valor ausente (NaN) encontrado.")
    else:
        print(" Valores ausentes encontrados:")
        print(tabulate(nulos.reset_index().rename(columns={"index": "Coluna", 0: "Total NaN"}), headers="keys", tablefmt="pretty"))

    simbolicos_invalidos = []
    for col in df.columns:
        count = df[col].astype(str).isin(VALORES_INVALIDOS).sum()
        if count > 0:
            simbolicos_invalidos.append((col, count))

    if not simbolicos_invalidos:
        print(" Nenhum valor simbólico inválido encontrado.")
    else:
        print("\n Valores simbólicos inválidos encontrados:")
        print(tabulate(simbolicos_invalidos, headers=["Coluna", "Quantidade"], tablefmt="pretty"))

    print("\n 2. Dimensões das imagens:")
    print("-" * 50)
    retangulares = df[df["largura"] != df["altura"]]
    if retangulares.empty:
        print(" Todas as imagens são quadradas (largura = altura).")
    else:
        primeiro = retangulares.iloc[0]
        print(f"  {len(retangulares)} imagens retangulares encontradas.")
        print(f"  Exemplo: arquivo={primeiro['arquivo']}, largura={primeiro['largura']}, altura={primeiro['altura']}")


    print("\n 3. Consistência dos dados de pixels:")
    print("-" * 50)
    df["soma_pixels"] = df["pixels_background"] + df["pixels_objeto"]
    inconsistencias = df[df["soma_pixels"] != df["total_pixels"]]

    if inconsistencias.empty:
        print(" Todas as imagens têm soma de pixels (background + objeto) igual ao total.")
    else:
        print(f" {len(inconsistencias)} imagens com soma de pixels inconsistente:")
        print(tabulate(inconsistencias[["arquivo", "pixels_background", "pixels_objeto", "total_pixels"]].head(), headers="keys", tablefmt="pretty"))

    df["soma_proporcao"] = df["proporcao_background"] + df["proporcao_objeto"]
    proporcoes_inconsistentes = df[abs(df["soma_proporcao"] - 1.0) > 0.01]
    if proporcoes_inconsistentes.empty:
        print(" Todas as imagens têm proporções que somam aproximadamente 1.0.")
    else:
        print(f" {len(proporcoes_inconsistentes)} imagens com proporções inconsistentes:")
        print(tabulate(proporcoes_inconsistentes[["arquivo", "proporcao_background", "proporcao_objeto", "soma_proporcao"]].head(), headers="keys", tablefmt="pretty"))

    print("\n 4. Verificação das faixas de porcentagem:")
    print("-" * 50)
    df["faixa_calculada"] = df["proporcao_objeto"].apply(formatar_porcentagem)
    faixas_incorretas = df[df["faixa_proporcao_objeto"] != df["faixa_calculada"]]
    if faixas_incorretas.empty:
        print(" Todas as faixas de porcentagem estão corretamente calculadas.")
    else:
        print(f" {len(faixas_incorretas)} imagens com faixas de porcentagem incorretas:")
        print(tabulate(faixas_incorretas[["arquivo", "proporcao_objeto", "faixa_proporcao_objeto", "faixa_calculada"]].head(), headers="keys", tablefmt="pretty"))

        # 5. Análise binária: presença de fundo e objeto
    print("\n 5. Análise binária (com base em background e objeto):")
    print("-" * 50)

    if {"somente_background", "somente_objeto", "background_e_objeto"}.issubset(df.columns):
        total = len(df)
        apenas_bg = df["somente_background"].sum()
        apenas_obj = df["somente_objeto"].sum()
        ambos = df["background_e_objeto"].sum()

        print(f" Imagens com apenas background: {apenas_bg} ({apenas_bg / total:.1%})")
        print(f" Imagens com apenas objeto: {apenas_obj} ({apenas_obj / total:.1%})")
        print(f" Imagens com background e objeto: {ambos} ({ambos / total:.1%})")

        if apenas_bg > 0:
            print("  Algumas imagens não têm nenhum pixel de objeto — podem ser irrelevantes para o modelo.")
        if apenas_obj > 0:
            print("  Algumas imagens têm apenas objeto — verifique se o fundo foi corretamente identificado como classe 0.")
    else:
        print(" As colunas 'somente_background', 'somente_objeto', 'background_e_objeto' não estão presentes no CSV.")
        print("Sem elas, não é possível avaliar a presença dos grupos binários nas máscaras.")


    print("\n" + "=" * 60)
    print(" RESUMO FINAL DA QUALIDADE DOS METADADOS")
    print("=" * 60)

    problemas = []
    if not nulos.empty:
        problemas.append(f"• Valores ausentes (NaN) em {len(nulos)} coluna(s)")
    if simbolicos_invalidos:
        problemas.append(f"• Valores simbólicos inválidos em {len(simbolicos_invalidos)} coluna(s)")
    if not retangulares.empty:
        problemas.append(f"• {len(retangulares)} imagens retangulares (largura ≠ altura)")
    if not inconsistencias.empty:
        problemas.append(f"• {len(inconsistencias)} imagens com soma de pixels inconsistente")
    if not proporcoes_inconsistentes.empty:
        problemas.append(f"• {len(proporcoes_inconsistentes)} imagens com proporções inconsistentes")
    if not faixas_incorretas.empty:
        problemas.append(f"• {len(faixas_incorretas)} imagens com faixas de porcentagem incorretas")
    if 'classes_presentes' in df.columns and classes_nao_mapeadas:
        problemas.append(f"• {len(classes_nao_mapeadas)} classe(s) não mapeadas no agrupamento binário")

    if problemas:
        print("\n PROBLEMAS ENCONTRADOS:")
        for problema in problemas:
            print(problema)
    else:
        print("\n METADADOS CONSISTENTES: Nenhum problema grave encontrado!")
        print("O dataset parece estar bem preparado para uso em modelos de segmentação semântica.")

if __name__ == "__main__":
    verificar_metadados(CSV_PATH)



ℹ️ Total de imagens no dataset: 400

 1. Valores ausentes ou inválidos:
--------------------------------------------------
 Nenhum valor ausente (NaN) encontrado.
 Nenhum valor simbólico inválido encontrado.

 2. Dimensões das imagens:
--------------------------------------------------
  400 imagens retangulares encontradas.
  Exemplo: arquivo=000.png, largura=960, altura=736

 3. Consistência dos dados de pixels:
--------------------------------------------------
 Todas as imagens têm soma de pixels (background + objeto) igual ao total.
 Todas as imagens têm proporções que somam aproximadamente 1.0.

 4. Verificação das faixas de porcentagem:
--------------------------------------------------
 Todas as faixas de porcentagem estão corretamente calculadas.

 5. Análise binária (com base em background e objeto):
--------------------------------------------------
 Imagens com apenas background: 0 (0.0%)
 Imagens com apenas objeto: 0 (0.0%)
 Imagens com background e objeto: 400 (100.0%)



Conclusão:

 Embora os metadados estejam íntegros e sem valores ausentes, sem erros de soma de pixels ou de proporções e com todas as faixas corretamente calculadas, há um ponto crítico: todas as 400 imagens são retangulares (960 × 736 no exemplo).

 E idealmente deveríamos trabalhar com formatos quadrados para manter a proporção dos pixels em tarefas de segmentação semântica. 

 Esse desvio pode afetar a performance de modelos que esperam entradas quadradas, portanto, uma solução seria redimensionar ou recortar as máscaras para um formato uniforme.